In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import xgboost as xgb
import numpy as np
import pandas as pd
from statistics import mean
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd
import xgboost as xgb
import optuna
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_log_error
import os
os.environ["LOKY_MAX_CPU_COUNT"] = "4" 

In [2]:
train_df = pd.read_csv('/kaggle/input/playground-series-s4e12/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s4e12/test.csv')

In [3]:
def date_trans(df):
    df['Policy Start Date']= pd.to_datetime(df['Policy Start Date'])
    df['Year'] = df['Policy Start Date'].dt.year
    df['Day'] = df['Policy Start Date'].dt.day
    df['Month'] = df['Policy Start Date'].dt.month
    df['DayOfWeek'] = df['Policy Start Date'].dt.dayofweek
    df.drop('Policy Start Date' , axis =1, inplace = True)
    return df

train_df = date_trans(train_df)
test_df = date_trans(test_df)

In [4]:
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

In [5]:
for col in train_df.select_dtypes(include='object').columns:
    train_df[col] = train_df[col].astype('category')
for col in test_df.select_dtypes(include='object').columns:
    test_df[col] = test_df[col].astype('category')

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import KBinsDiscretizer
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, mean_squared_log_error

def log_transform(y):
    """
    Apply log transformation safely, handling zero and negative values
    """
    return np.log1p(y)

def inverse_log_transform(y_log):
    """
    Revert log transformation
    """
    return np.expm1(y_log)

# Example dataset (replace with your dataset)
np.random.seed(42)
df = train_df

# Find the target column (assumes it contains 'Premium' or 'Amount')
target_column = [col for col in df.columns if 'premium' in col.lower() or 'amount' in col.lower()]
if not target_column:
    raise ValueError("Could not find target column. Please specify the column name for premium/amount.")
target_column = target_column[0]

# Identify column types
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = df.select_dtypes(include=['object', 'category']).columns.tolist()

# Remove target variable from features
if target_column in numeric_features:
    numeric_features.remove(target_column)
if target_column in categorical_features:
    categorical_features.remove(target_column)

# Features and target
X = df.drop(columns=[target_column])
y = df[target_column]

# Log transform the target variable
y_log = log_transform(y)

# Stratify target by binning into discrete intervals
binner = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
y_binned = binner.fit_transform(y_log.values.reshape(-1, 1)).astype(int).ravel()

# Stratified K-Fold
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Arrays to store predictions
oof_predictions_log = np.zeros(len(X))  # Out-of-fold predictions in log space
oof_predictions = np.zeros(len(X))  # Out-of-fold predictions in original space

# Test set (replace with your actual test set)
X_test = test_df

# Prepare for cross-validation
test_preds_per_fold_log = np.zeros((len(X_test), n_splits))  # Store test predictions per fold in log space
test_preds_per_fold = np.zeros((len(X_test), n_splits))  # Store test predictions per fold in original space
fold_oof_results = []  # To store ID, target, and OOF predictions

# LightGBM Regressor parameters
model_params = {
    'objective': 'regression',
    'metric': 'rmsle',
    'random_state': 42,
    'is_unbalance': True,
      'nan_as_missing': True,
      'n_estimators': 2900, 
      'learning_rate': 0.009292502487271825,
      'max_depth': 11,
      'num_leaves': 88, 
      'min_child_samples': 46,
      'min_child_weight': 0.0008888702484611456, 
      'subsample': 0.8552895795397466, 
      'subsample_freq': 5, 
      'colsample_bytree': 0.9135439280459978,
      'reg_alpha': 0.0021100979917979567,
      'reg_lambda': 5.0153235478869446e-08,
      'verbose': -1,
        'device': 'gpu',
        'gpu_platform_id': 0,
        'gpu_device_id': 0,
        'gpu_use_dp': True  
}

# Cross-validation loop
for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y_binned)):
    print(f"Fold {fold + 1}/{n_splits}")
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    y_train_log, y_valid_log = y_log.iloc[train_idx], y_log.iloc[valid_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
    
    # Model with early stopping on log-transformed target
    model = LGBMRegressor(**{**model_params, 'verbose': -1})
    model.fit(X_train, y_train_log, 
              eval_set=[(X_valid, y_valid_log)],
              )
    
    # Predictions in log space
    oof_predictions_log[valid_idx] = model.predict(X_valid)
    
    # Convert log predictions back to original space
    oof_predictions[valid_idx] = inverse_log_transform(oof_predictions_log[valid_idx])

    fold_rmsle = rmsle(y.iloc[valid_idx], oof_predictions[valid_idx])
    print(f"Fold {fold + 1} RMSLE: {fold_rmsle:.4f}")
    
    # Store fold results with ID, target, and OOF predictions
    fold_result = pd.DataFrame({
        'ID': X.index[valid_idx],       # Assuming X has an index as IDs
        'Actual': y.iloc[valid_idx],    # Actual target values
        'OOF_Pred_LGB': oof_predictions[valid_idx],    # OOF predictions
        'Fold': fold + 1               # Fold number
    })
    fold_oof_results.append(fold_result)
    
    # Test set predictions for this fold
    test_preds_per_fold_log[:, fold] = model.predict(X_test)
    test_preds_per_fold[:, fold] = inverse_log_transform(test_preds_per_fold_log[:, fold])

# Combine OOF results
oof_results_df = pd.concat(fold_oof_results, axis=0, ignore_index=True)

# Average predictions on test data
final_test_predictions = test_preds_per_fold.mean(axis=1)

# Evaluate OOF predictions
oof_mse = mean_squared_error(y, oof_predictions)
oof_rmsle = np.sqrt(mean_squared_log_error(y, oof_predictions))

print(f"OOF Mean Squared Error: {oof_mse:.4f}")
print(f"OOF Root Mean Squared Log Error: {oof_rmsle:.4f}")

print("Final Test Predictions:", final_test_predictions)
print(oof_results_df.head())

Fold 1/5
Fold 1 RMSLE: 1.0475
Fold 2/5
Fold 2 RMSLE: 1.0463
Fold 3/5
Fold 3 RMSLE: 1.0456
Fold 4/5
Fold 4 RMSLE: 1.0463
Fold 5/5
Fold 5 RMSLE: 1.0457
OOF Mean Squared Error: 852315.9082
OOF Root Mean Squared Log Error: 1.0463
Final Test Predictions: [718.52837997 863.28479822 799.29849307 ... 870.0892496  790.86534285
 790.92789258]
   ID  Actual  OOF_Pred_LGB  Fold
0   4  2022.0    790.468576     1
1  15   849.0    844.729374     1
2  21  2670.0    695.666440     1
3  30   641.0    835.328327     1
4  34  2152.0    791.740481     1


In [7]:
sub = pd.read_csv('/kaggle/input/playground-series-s4e12/sample_submission.csv')
sub['Premium Amount'] = final_test_predictions
sub.to_csv('lgsubmission.csv', index=False)
sub.head()
oof_results_df.to_csv('oof_lgbm.csv',index = False)

In [8]:
sub.head()

,id,Premium Amount
0,1200000,718.528380
1,1200001,863.284798
2,1200002,799.298493
3,1200003,789.131510
4,1200004,778.381459
